***INFERENCE SESSION***

In [1]:
import numpy as np
import librosa
from tensorflow.keras.models import load_model
inf_model = load_model('D:/SPEECH_RECOG/all_speech_recog.h5')
tr_mean = np.load('tr_mean.npy')
tr_std = np.load('tr_std.npy')

In [2]:
inf_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 13, 73, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 6, 36, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 6, 36, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 2, 33, 32)      │        32,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 1, 16, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1, 16, 32)      │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,225 (426.66 KB)

 Trainable params: 109,031 (425.90 KB)

 Non-trainable params: 192 (768.00 B)

 Optimizer params: 2 (12.00 B)

In [3]:
def resize_array(array):
    new_matrix = np.zeros((30,150))   
    for i in range(30):               
        for j in range(150):          
            try:                                
                new_matrix[i][j] = array[i][j]
            except IndexError:                  
                pass
    return new_matrix

In [4]:
def inf(audio_path):
    z, sz = librosa.load(audio_path, sr=16000)
    mfc=librosa.feature.mfcc(y=z, sr=sz, fmin=50, n_mfcc=30)
    rez_mfc=resize_array(mfc)
    rez_mfc=np.array(rez_mfc)
    mfcc_standardized = (rez_mfc - tr_mean) / tr_std
    mfcc_standardized = mfcc_standardized[..., np.newaxis]
    mfcc_standardized = mfcc_standardized[np.newaxis, ...]
    predictions = inf_model.predict(mfcc_standardized)
    predicted_emotion = np.argmax(predictions, axis=1)[0]
    emotion_map = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
    predicted_emotion_label = emotion_map[predicted_emotion]
    return(predicted_emotion_label)

In [5]:
import time
from IPython.display import Audio
file_dir="D:/SPEECH_RECOG/RAVDESS/Actor_02/03-01-01-01-02-02-02.wav"
start=time.time()
print(inf(file_dir))
end=time.time()
print("Inference time : ",(end-start))
Audio(file_dir)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
neutral
Inference time :  3.9132893085479736


In [7]:
pip install moviepy

     ---------------------------------------- 0.0/388.3 kB ? eta -:--:--
     - -------------------------------------- 10.2/388.3 kB ? eta -:--:--
     ------ ------------------------------ 71.7/388.3 kB 787.7 kB/s eta 0:00:01
     -------------------------------------- 388.3/388.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for decorator<5.0,>=4.0.2 from https://files.pythonhosted.org/packages/ed/1b/72a1821152d07cf1d8b6fce298aeb06a7eb90f4d6d41acec9861e7cc6df0/decorator-4.4.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for imageio<3.0,>=2.5 from https://files.pythonhosted.org/packages/3d/84/f1647217231f6cc46883e5d26e870cc3e1520d458ecd52d6df750810d53c/imageio-2.34.2-py3-none-any.whl.metadata
  Obtaining dependency information for imageio_ffmpeg>=0.2.0 from https://files.pythonhosted.org/packages/a9/1c/1b9c72bf839def47626436ea5ebaf643404f7850482c5fafd7


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
from moviepy.editor import VideoFileClip

# Define the input video file and output audio file
mp4_file = "D:/SPEECH_RECOG/video/emotions.mp4"
mp3_file = "D:/SPEECH_RECOG/audio/extracted_audio.mp3"

# Load the video clip
video_clip = VideoFileClip(mp4_file)

# Extract the audio from the video clip
audio_clip = video_clip.audio
# Write the audio to a separate file
audio_clip.write_audiofile(mp3_file)

# Close the video and audio clips
audio_clip.close()
video_clip.close()
Audio('D:/SPEECH_RECOG/audio/extracted_audio.mp3')

MoviePy - Writing audio in D:/SPEECH_RECOG/audio/extracted_audio.mp3


MoviePy - Done.


In [40]:
from moviepy.editor import AudioFileClip
import os

# Input MP3 file and output directory
input_file = "D:/SPEECH_RECOG/audio/extracted_audio.mp3"
segment_duration = 3  # duration in seconds
output_dir = "D:/SPEECH_RECOG/audio"
os.makedirs(output_dir, exist_ok=True)

# Load the MP3 file
audio = AudioFileClip(input_file)

# Get the total duration of the audio file
audio_duration = audio.duration

# Split the audio into 3-second segments and save as WAV files
start_time = 0
segment_index = 0

dict={}
while start_time < audio_duration:
    end_time = min(start_time + segment_duration, audio_duration)
    segment = audio.subclip(start_time, end_time)
    segment_file = os.path.join(output_dir, f"segment.wav")
    segment.write_audiofile(segment_file, codec='pcm_s16le')
    prediction = inf(segment_file)
    print(f"Prediction for the video in the duration {start_time}:{end_time} => {prediction}")
    dict[f"{start_time}:{end_time}"]=prediction
    start_time += segment_duration

# Clean up: Close the audio file
audio.close()


MoviePy - Writing audio in D:/SPEECH_RECOG/audio\segment.wav


MoviePy - Done.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Prediction for the video in the duration 0:3 => sad
<IPython.lib.display.Audio object>
MoviePy - Writing audio in D:/SPEECH_RECOG/audio\segment.wav


MoviePy - Done.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Prediction for the video in the duration 3:6 => sad
<IPython.lib.display.Audio object>
MoviePy - Writing audio in D:/SPEECH_RECOG/audio\segment.wav


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction for the video in the duration 30:33 => disgust
<IPython.lib.display.Audio object>
MoviePy - Writing audio in D:/SPEECH_RECOG/audio\segment.wav


MoviePy - Done.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction for the video in the duration 33:36 => fear
<IPython.lib.display.Audio object>
MoviePy - Writing audio in D:/SPEECH_RECOG/audio\segment.wav


MoviePy - Done.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Prediction for the video in the duration 36:38.06 => sad
<IPython.lib.display.Audio object>


In [41]:
dict

{'0:3': 'sad',
 '3:6': 'sad',
 '6:9': 'sad',
 '9:12': 'sad',
 '12:15': 'disgust',
 '15:18': 'fear',
 '18:21': 'sad',
 '21:24': 'sad',
 '24:27': 'sad',
 '27:30': 'sad',
 '30:33': 'disgust',
 '33:36': 'fear',
 '36:38.06': 'sad'}